In [ ]:
# importing packages and libaray

import matplotlib.pyplot as plt
import numpy as np
from numpy import zeros
from numpy import ones
import os
import PIL
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
import importlib
import Ganmodel
from tensorflow.keras.layers import Conv2D, MaxPooling2D,LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input

In [ ]:
# Display generated images for a display interval every 1000 epochs

def display_images(n): 
        r, c = 1,4
        noise = np.random.normal(0, 1, (r * c,100)) 
        generated_images = generator.predict(noise) 
      
        imgs = generator.predict(noise)
        fig = plt.figure(figsize=(40,10))
        for i, img in enumerate(generated_images):
            ax = fig.add_subplot(1,4,i+1)
            ax.imshow(img,cmap="gray")
        fig.suptitle("Generated images: %s"%n,fontsize=30)

In [ ]:
# function for loading images.

def data_gen(img_folder, batch_size):
    c = 0
    n = os.listdir(img_folder) #List of training images
    np.random.shuffle(n)
    print("Number of Train_images:",len(n))
    
    while (True):
        img = np.zeros((batch_size, 256, 256, 1)).astype('float')
       
         #Reading set of images for a mentioned batch-size.
        for i in range(c, c+batch_size):
            train_img = tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(
                img_folder+'/'+n[i],color_mode='grayscale', target_size=(256, 256)))/255
            
            #add to array - img[0], img[1], and so on.
            img[i-c] = train_img 
             
        c+=batch_size
        if(c+batch_size>=len(os.listdir(img_folder))):
            c=0
            np.random.shuffle(n)
                  
        return img

In [ ]:
# define the combined generator and discriminator model, for updating the generator

def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', 
                         optimizer=tf.keras.optimizers.Adam(0.0002,0.5))
    return model

In [ ]:
# select real image samples

def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    # select images
    X = dataset[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return X, y

In [ ]:
# generate points in latent space as input for the generator

def generate_noise_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = np.random.normal(0, 1, (n_samples, 100))
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [ ]:
# generate fake image samples

def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_noise_points(latent_dim, n_samples)
    # predict outputs
    X = generator.predict(x_input)
    # create class labels
    y = zeros((n_samples, 1))
    return X, y

In [ ]:
def plot_loss(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
    # plot Discrimination loss
    plt.subplot(2, 1, 1)
    plt.plot(d1_hist, label='d-real_loss')
    plt.plot(d2_hist, label='d-fake_loss')
    plt.legend()
    
    # plot Generator loss
    plt.subplot(2, 1, 2)
    plt.plot(g_hist, label='gen_loss')
    plt.legend()
    
    plt.show()
    
    #plt.subplot(3, 1, 3)
    plt.plot(d1_hist, label='d-real_loss')
    plt.plot(d2_hist, label='d-fake_loss')
    plt.plot(g_hist, label='gen_loss')
    plt.legend()
    
    plt.show()

In [ ]:
# defining path for accessing dataset
image_dir = 'H:/Downloads/keras_png_slices_data/Dataset/keras_png_slices_train/image'

#Parameters
num_epochs=15000
batch_size=16
display_interval=1000
train_gen = data_gen(image_dir, batch_size = 16)

#Building Model
generator = Ganmodel.make_generator_model()
discriminator = Ganmodel.make_discriminator_model()
combined_network = define_gan(generator, discriminator)

# prepare lists for storing stats each iteration
d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()

              
# calculate the number of batches per epoch
bat_per_epo = int(train_gen.shape[0] / batch_size)

# calculate the total iterations based on batch and epoch
n_steps = bat_per_epo * num_epochs

for i in range(n_steps+1):             

            #Sampling a random half of images and Training the discriminator
            X_real, y_real = generate_real_samples(train_gen, int(batch_size / 2))
            d_loss1, d_acc1 = discriminator.train_on_batch(X_real, y_real)
            
            #Sampling noise, generating a batch of new fake images and Training the discriminator
            X_fake, y_fake = generate_fake_samples(generator, 100, int(batch_size / 2))
            d_loss2, d_acc2 = discriminator.train_on_batch(X_fake, y_fake)
               
            #Training the generator to generate images which pass the authenticity test 
            X_gan = generate_noise_points(100, batch_size)
            y_gan = ones((batch_size, 1))
            g_loss = combined_network.train_on_batch(X_gan, y_gan) 
            
            #Tracking the progress                 
            if i % display_interval == 0:
                #print("Value of i:", i,"Value of Display_Interval: ", display_interval)
                display_images(i) 
                    
            # record history
            d1_hist.append(d_loss1)
            d2_hist.append(d_loss2)
            g_hist.append(g_loss)
            a1_hist.append(d_acc1)
            a2_hist.append(d_acc2)
                   


In [ ]:
# Plot Training Loss
plot_loss(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)